## Leassons learned:
    
1) It's possible to use numpy to compute value functions, policy iteration, etc
    * Upsides:
        - Will run much more faster
    * Downsides:
        - As the execution is vectorized, we can't do in-place operations, so its have a slower convergence. But probably the speed gained using vectors.
        - Need to precompute everything previously. Withtout vectors we can calculate values on the fly.

2) Is a good a idea to check the total probability per action/state/action (check sum(axis=2)). Each cell must be 1

3) Also is good idea to check the result of (probability * policy).sum(axis=1). Each row a total value of 1

4) To use a function like probability(state, action, new_state), while calculating the rewards per new_state, that is already rated by the probatility, we have to divied the cost of the action (moving cars) by the number of states, like distributing the cost across new_states because the reward is already distributed?
    
5) The function for policy_improvement and policy_evaluation is changed for this problems as the probability is already applied to the reward, so the reward have to be excluded from the multiplication

In [147]:
import numpy as np

In [148]:

# policy * prob * (reward + gamma * value)

actions = range(-2, 2)
states = range(0, 10)

prob = np.full([len(states), len(actions), len(states)], 1/(len(states)))
policy = np.full([len(states), len(actions), 1], 1/len(actions))

#(prob * policy).sum(axis=2).sum(axis=1)
(policy.reshape(10, 4) * prob.sum(axis=2))
#prob
prob.sum(axis=2)
# sum(p(s', r | s, a), s', r) = 1

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])

In [149]:
nums = [poisson_distribution(i, 3) for i in range(4)]
print(nums)
print(sum(nums))

[0.04978706836786395, 0.14936120510359185, 0.22404180765538778, 0.22404180765538778]
0.6472318887822314


In [150]:
from math import factorial, e

poisson_dict = {}
def poisson_prob(n, lb):
    key = (n, lb)
    if key in poisson_dict: return poisson_dict[key]
    value = (pow(lb, n)/factorial(n))*(pow(e, -lb))
    poisson_dict[key] = value
    return value

loc_rental_probability_cache = {}
def loc_rental_probability(start, end, rent_lam, return_lam):
    """
    calculate the rental probabilities and rewards given a particular start and end count
    :param start: the day start count
    :param end: the day end count
    :param rent_lam: the rental poisson lambda
    :param return_lam: the return poisson lambda
    :return: a list of probability/reward tuples enumerating the possible outcomes.
    """
    key = (start, end, rent_lam, return_lam)
    if key in loc_rental_probability_cache: return loc_rental_probability_cache[key]
    
    rv = []
    for rented in range(0, start+1):
        remaining = start - rented
        returned = end - remaining
        if returned >= 0: # feasible
            if remaining == 0:
                # cars that can be rented is capped at the number on the lot, but in theory an infinite number of customers could arrive...
                prob = 1
                for i in range(0, rented):
                    prob -= poisson_prob(i, rent_lam)
            else:
                prob = poisson_prob(rented, rent_lam)
            if end == 20:
                # lot size is capped at 20, but in theory an infinite number of cars could be returned...
                return_prob = 1
                for i in range(0, returned):
                    return_prob -= poisson_prob(i, return_lam)
            else:
                return_prob = poisson_prob(returned, return_lam)

            prob *= return_prob
            reward = 10 * rented
            rv.append((prob, reward))

    loc_rental_probability_cache[key] = rv
    return rv

def probability_reward(state, action, new_state):
    """
    calculate the day activity (rental/return activity) probabilities
    :param start_state: the start of day state
    :return: a next_state -> (transition probability, *expected* reward) tuples.
    """
    rv = {}
    
    state = (min(20, state[0] - action), min(20, state[1] + action))
    
    l1_probs = loc_rental_probability(state[0], new_state[0], 3, 3)
    l2_probs = loc_rental_probability(state[1], new_state[1], 4, 2)
    prob = 0
    reward = 0
    for l1p, l1r in l1_probs:
        for l2p, l2r in l2_probs:
            p = l1p*l2p
            prob += p
            reward += p * (l1r+l2r)

    #print("returning for ", (state, action, new_state), "prob:", prob)
    return (prob, reward - abs(action)*2/441)

class Rental():
    actions = np.array(range(-5, 6))
    states = np.array([(i,j) for i in range(21) for j in range(21)])
    
    def __init__(self):
        self.num_actions = len(self.actions)
        self.num_states = len(self.states)
        
        self.probabilities = np.full([self.num_states, self.num_actions, self.num_states], 0, dtype=np.float32)
        self.rewards = np.full([self.num_states, self.num_actions, self.num_states], 0, dtype=np.float32)
        
        for state_idx, state in enumerate(self.states):
            for action_idx, action in enumerate(self.actions):
                for new_state_idx, new_state in enumerate(self.states):
                    p,r = probability_reward(state, action, new_state)
                    self.probabilities[state_idx][action_idx][new_state_idx] = p
                    self.rewards[state_idx][action_idx][new_state_idx] = r
    
r = Rental()

In [151]:
value_state = np.full(r.num_states, 0)
z = np.zeros(r.num_actions)
z[5] = 1
policy = np.full([r.num_states, len(r.actions)], z).reshape(r.num_states, len(r.actions), 1)

def policy_eval(policy, value_state, env, discount_factor=1.0, theta=0.00001):
    # while True:
    for i in range(1000):
        delta = 0
        old_value_state = value_state
        
        # new_v_action_state = (value_state*discount_factor + env.rewards)*env.probabilities*policy
        new_v_action_state = (env.rewards + (value_state*discount_factor)*env.probabilities)*policy
        new_v = new_v_action_state.sum(axis=2).sum(axis=1)
        value_state = new_v
        
        delta = max(delta, np.abs(old_value_state - value_state).max())
        if delta < theta:
            print(i)
            break

    return value_state

discount_factor=0.9
theta=1

def policy_iteration(policy, value_state, env):
    value_state = policy_eval(policy, value_state, env, discount_factor=discount_factor, theta=theta)
    
    # new_policy = (env.probabilities * (env.rewards + discount_factor*value_state)).sum(axis=2).argmax(axis=1)
    new_policy = (env.rewards + env.probabilities * (discount_factor*value_state)).sum(axis=2).argmax(axis=1)
    new_policy = new_policy.reshape(new_policy.shape[0], 1)
    return new_policy, value_state

def print_policy(policy):
    pol = (policy.reshape(441, 11).argmax(axis=1) - 5).reshape(21, 21)
    print("    ", " ".join(map(lambda x: "%2d" % x, range(0, 21))))
    print("    ---------------------------------------------------------------")
    for row in range(20, -1, -1):
        line = []
        for col in range(0, 21):
            line.append(pol[row][col])
        print("%2d" % row , "|", " ".join(map(lambda x: "%2d" % x, line)))

for i in range(10):
    new_policy_argmax, value_state = policy_iteration(policy, value_state, r)
    new_policy = np.zeros([r.num_states, len(r.actions)])
    new_policy[range(len(policy)), new_policy_argmax.reshape(new_policy_argmax.shape[0],)] = 1
    new_policy = new_policy.reshape([r.num_states, len(r.actions), 1])

    print("policy (0, 0):", r.actions[new_policy[0].argmax()])
    
    diff = np.abs(policy - new_policy).max(axis=1)
    old_policy = policy
    policy = new_policy
    
    if diff[diff != 0].shape[0] != 0:
        pass
    else:
        break

#policy.reshape(16,4).argmax(axis=1).reshape(4, 4), value_state.reshape(4,4)


37
policy (0, 0): 0
8
policy (0, 0): 0
8
policy (0, 0): 0
1
policy (0, 0): 0
0
policy (0, 0): 0
0
policy (0, 0): 0


In [152]:
print_policy(policy)

      0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
    ---------------------------------------------------------------
20 |  5  5  5  5  4  4  3  3  3  3  2  2  2  2  1  1  1  0  0  0  0
19 |  5  5  5  4  4  3  3  2  2  2  2  1  1  1  1  0  0  0  0  0  0
18 |  5  5  5  4  3  3  2  2  1  1  1  1  0  0  0  0  0  0  0  0  0
17 |  5  5  5  4  3  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0
16 |  5  5  5  4  3  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0
15 |  5  5  5  4  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
14 |  5  5  4  4  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
13 |  5  5  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
12 |  5  5  4  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
11 |  5  4  4  3  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
10 |  4  4  3  3  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 9 |  4  3  3  2  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 8 |  3  3  2  2  1  1  0  0  0  0  0  0  0  0  

In [153]:
value_state

array([ 418.22063631,  428.19284986,  438.04847796,  447.6536038 ,
        456.88601576,  465.67745605,  474.01477469,  481.91709186,
        489.78028008,  497.27415866,  504.6856609 ,  511.78878111,
        518.54292603,  525.14124393,  531.53119735,  537.61675891,
        543.41774259,  548.95081598,  554.36953493,  559.56870546,
        564.5137228 ,  428.09433812,  438.0664838 ,  447.92177827,
        457.52606082,  466.75696194,  475.54625776,  483.88094855,
        491.78028038,  499.27415878,  506.68566115,  513.7887811 ,
        520.54292594,  527.14124436,  533.53119698,  539.61675942,
        545.4177427 ,  550.95081542,  556.36953553,  561.56870589,
        566.51372308,  571.19572038,  437.56766346,  447.53950134,
        457.39326746,  466.99364952,  476.21760798,  484.997119  ,
        493.31993601,  501.20604442,  508.68566094,  515.78878113,
        522.54292629,  529.14124384,  535.53119738,  541.61675882,
        547.4177426 ,  552.95081543,  558.36953525,  563.56870